
## 경로 및 패키지 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/tag'

In [ ]:
import pandas as pd
import gensim, logging
import multiprocessing
import os

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
pd.options.display.max_info_columns =200
pd.options.display.max_columns = 200
pd.options.display.max_info_rows =100
pd.options.display.max_rows = 100

import os
from tqdm import tqdm_notebook

## 데이터 전처리

- 크롤링한 raw data
- 태그 + 제목
- 특수문자 제거

In [ ]:
#내 작업
query_list = ['갤럭시', '마우스']

In [ ]:
total_df = []
for i in range(len(query_list)):
    total_df.append(pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/tag/{query_list[i]}.csv'))

In [ ]:
keyboard = total_df[0].copy()

In [ ]:
keyboard

,product_id,title,keyword,cat1,cat2,cat3,view,image
0,173770536,미개봉 갤럭시워치4 40mm 실버 블루투스,"갤럭시,워치,워치4,40mm,watch",디지털/가전,모바일,웨어러블(워치),175,https://media.bunjang.co.kr/product/173770536_...
1,174570492,갤럭시 구스 105,105,남성의류,패딩/점퍼,기타(패딩/점퍼),0,https://media.bunjang.co.kr/product/174570492_...
2,173979801,갤럭시 s20 플러스 쥐색 회색,"갤럭시,s20,플러스",디지털/가전,모바일,스마트폰,142,https://media.bunjang.co.kr/product/173979801_...
3,174570489,삼성 갤럭시S21울트라 실버 256GB 판매합니다 64570,NaN,디지털/가전,모바일,스마트폰,0,https://media.bunjang.co.kr/product/174570489_...
4,174441195,(미개봉)갤럭시워치4 그린 44mm,"미밴드,애플워치,갤럭시워치,순토",디지털/가전,모바일,웨어러블(워치),20,https://media.bunjang.co.kr/product/174441195_...
...,...,...,...,...,...,...,...,...
5995,174499653,갤럭시S8플러스 64GB 핑크 기기 판매합니다 8459,"갤럭시S8플러스,S8플러스핑크,갤럭시S8+,갤럭시S8+핑크,갤럭시S8+중고",디지털/가전,모바일,스마트폰,9,https://media.bunjang.co.kr/product/174499653_...
5996,172541108,#특가# 갤럭시워치3 41mm R850 블루투스전용 [1209],"갤럭시워치3,워치3,워치341mm,갤럭시워치341m,r850",디지털/가전,모바일,스마트폰,58,https://media.bunjang.co.kr/product/172541108_...
5997,174190990,#특가# 갤럭시S10 5G 256GB 블랙 [1225],"갤럭시S105G,S105G,S105G블랙,S105G중고,갤럭시S10",디지털/가전,모바일,스마트폰,50,https://media.bunjang.co.kr/product/174190990_...
5998,174189909,#특가# 갤럭시Z폴드2 256GB 블랙 [무잔상] 1225,"갤럭시Z폴드2,폴드2,폴드2블랙,폴드2공기계,폴드2중고",디지털/가전,모바일,스마트폰,102,https://media.bunjang.co.kr/product/174189909_...


In [ ]:
for i in range(5):
  keyboard[f'tag{i}'] = keyboard.keyword.str.split(',').str[i]

In [ ]:
keyboard = keyboard.fillna(0)
keyboard

,product_id,title,keyword,cat1,cat2,cat3,view,image,tag0,tag1,tag2,tag3,tag4
0,173770536,미개봉 갤럭시워치4 40mm 실버 블루투스,"갤럭시,워치,워치4,40mm,watch",디지털/가전,모바일,웨어러블(워치),175,https://media.bunjang.co.kr/product/173770536_...,갤럭시,워치,워치4,40mm,watch
1,174570492,갤럭시 구스 105,105,남성의류,패딩/점퍼,기타(패딩/점퍼),0,https://media.bunjang.co.kr/product/174570492_...,105,0,0,0,0
2,173979801,갤럭시 s20 플러스 쥐색 회색,"갤럭시,s20,플러스",디지털/가전,모바일,스마트폰,142,https://media.bunjang.co.kr/product/173979801_...,갤럭시,s20,플러스,0,0
3,174570489,삼성 갤럭시S21울트라 실버 256GB 판매합니다 64570,0,디지털/가전,모바일,스마트폰,0,https://media.bunjang.co.kr/product/174570489_...,0,0,0,0,0
4,174441195,(미개봉)갤럭시워치4 그린 44mm,"미밴드,애플워치,갤럭시워치,순토",디지털/가전,모바일,웨어러블(워치),20,https://media.bunjang.co.kr/product/174441195_...,미밴드,애플워치,갤럭시워치,순토,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,174499653,갤럭시S8플러스 64GB 핑크 기기 판매합니다 8459,"갤럭시S8플러스,S8플러스핑크,갤럭시S8+,갤럭시S8+핑크,갤럭시S8+중고",디지털/가전,모바일,스마트폰,9,https://media.bunjang.co.kr/product/174499653_...,갤럭시S8플러스,S8플러스핑크,갤럭시S8+,갤럭시S8+핑크,갤럭시S8+중고
5996,172541108,#특가# 갤럭시워치3 41mm R850 블루투스전용 [1209],"갤럭시워치3,워치3,워치341mm,갤럭시워치341m,r850",디지털/가전,모바일,스마트폰,58,https://media.bunjang.co.kr/product/172541108_...,갤럭시워치3,워치3,워치341mm,갤럭시워치341m,r850
5997,174190990,#특가# 갤럭시S10 5G 256GB 블랙 [1225],"갤럭시S105G,S105G,S105G블랙,S105G중고,갤럭시S10",디지털/가전,모바일,스마트폰,50,https://media.bunjang.co.kr/product/174190990_...,갤럭시S105G,S105G,S105G블랙,S105G중고,갤럭시S10
5998,174189909,#특가# 갤럭시Z폴드2 256GB 블랙 [무잔상] 1225,"갤럭시Z폴드2,폴드2,폴드2블랙,폴드2공기계,폴드2중고",디지털/가전,모바일,스마트폰,102,https://media.bunjang.co.kr/product/174189909_...,갤럭시Z폴드2,폴드2,폴드2블랙,폴드2공기계,폴드2중고


In [ ]:
len(keyboard['tag2'].unique())

1564

In [ ]:
keylist = []

for i in range(6000):
    for j in range(5):
        if keyboard[f'tag{j}'][i] != 0:
            keylist.append(keyboard[f'tag{j}'][i] + ' ' + keyboard['title'][i])
        else:
            pass

keylist

['갤럭시 미개봉 갤럭시워치4 40mm 실버 블루투스',
 '워치 미개봉 갤럭시워치4 40mm 실버 블루투스',
 '워치4 미개봉 갤럭시워치4 40mm 실버 블루투스',
 '40mm 미개봉 갤럭시워치4 40mm 실버 블루투스',
 'watch 미개봉 갤럭시워치4 40mm 실버 블루투스',
 '105 갤럭시   구스 105',
 '갤럭시 갤럭시 s20 플러스 쥐색 회색',
 's20 갤럭시 s20 플러스 쥐색 회색',
 '플러스 갤럭시 s20 플러스 쥐색 회색',
 '미밴드 (미개봉)갤럭시워치4 그린 44mm ',
 '애플워치 (미개봉)갤럭시워치4 그린 44mm ',
 '갤럭시워치 (미개봉)갤럭시워치4 그린 44mm ',
 '순토 (미개봉)갤럭시워치4 그린 44mm ',
 '버즈 갤럭시 버즈 프로 미개봉 새제품 판매합니다',
 '버즈프로 갤럭시 버즈 프로 미개봉 새제품 판매합니다',
 '버즈2 갤럭시 버즈 프로 미개봉 새제품 판매합니다',
 '버즈라이브 갤럭시 버즈 프로 미개봉 새제품 판매합니다',
 '에어팟 갤럭시 버즈 프로 미개봉 새제품 판매합니다',
 '폴더2 삼성 갤럭시 폴더2 G160 새제품 판매합니다',
 'G160 삼성 갤럭시 폴더2 G160 새제품 판매합니다',
 '폴더 삼성 갤럭시 폴더2 G160 새제품 판매합니다',
 '폴더폰 삼성 갤럭시 폴더2 G160 새제품 판매합니다',
 '효도폰 삼성 갤럭시 폴더2 G160 새제품 판매합니다',
 '갤럭시A40 갤럭시A905G A908 LG 화이트 128GB 17만 47695',
 '갤럭시A50 갤럭시A905G A908 LG 화이트 128GB 17만 47695',
 '갤럭시J4플러스 갤럭시A905G A908 LG 화이트 128GB 17만 47695',
 '갤럭시J6 갤럭시A905G A908 LG 화이트 128GB 17만 47695',
 '갤럭시A90 갤럭시A905G A908 LG 화이트 128GB 17만 47695',
 '워치 겔럭시기어 워치4 클래식 46mm 팔아요!!',
 '겔럭시기어 겔럭시기어 워치

In [ ]:
kdf = pd.DataFrame(keylist, columns = ['merge'])

In [ ]:
kdf

,merge
0,갤럭시 미개봉 갤럭시워치4 40mm 실버 블루투스
1,워치 미개봉 갤럭시워치4 40mm 실버 블루투스
2,워치4 미개봉 갤럭시워치4 40mm 실버 블루투스
3,40mm 미개봉 갤럭시워치4 40mm 실버 블루투스
4,watch 미개봉 갤럭시워치4 40mm 실버 블루투스
...,...
19593,폴드2 #특가# 갤럭시Z폴드2 256GB 블랙 [무잔상] 1225
19594,폴드2블랙 #특가# 갤럭시Z폴드2 256GB 블랙 [무잔상] 1225
19595,폴드2공기계 #특가# 갤럭시Z폴드2 256GB 블랙 [무잔상] 1225
19596,폴드2중고 #특가# 갤럭시Z폴드2 256GB 블랙 [무잔상] 1225


In [ ]:
from typing import List

import pandas as pd
import itertools
import re
 
def load_sentences(df: pd.DataFrame = None) -> List[str]:
    """
    개별 뉴스에서 </p>와 <p>로 구분되어 있는 여러 문장을 문장으로 쪼갬
    :param df: 전처리 대상 데이터 프레임
    :return: 문장이 분리된 데이터 프레임
    """
    all_sentences = []
    for news in kdf['merge'].values:
        sentences_in_news = news.split(' </p> <p> ')
        sentences_in_news = [x.replace('</p>', '').replace('<p>','').strip() for x in sentences_in_news]
        all_sentences.append(sentences_in_news)

    merged = list(itertools.chain(*all_sentences))

    return merged

In [ ]:
train_sentences = load_sentences(kdf)

In [ ]:
stoplist = ['(', ')', '', '', '', '', '', '', ]

In [ ]:
for i in range(len(train_sentences)):
    train_sentences[i] = train_sentences[i].replace('(', ' ')
    train_sentences[i] = train_sentences[i].replace(')', ' ')
    train_sentences[i] = train_sentences[i].replace('^', ' ')
    train_sentences[i] = train_sentences[i].replace('*', ' ')
    train_sentences[i] = train_sentences[i].replace('#', ' ')
    train_sentences[i] = train_sentences[i].replace('&', ' ')
    train_sentences[i] = train_sentences[i].replace('[', ' ')
    train_sentences[i] = train_sentences[i].replace(']', ' ')
    train_sentences[i] = train_sentences[i].replace('!', ' ')
    train_sentences[i] = train_sentences[i].replace('?', ' ')
    train_sentences[i] = train_sentences[i].replace('.', ' ')
    #train_sentences[i] = train_sentences[i].replace('>', '')


In [ ]:
train_sentences

['갤럭시 미개봉 갤럭시워치4 40mm 실버 블루투스',
 '워치 미개봉 갤럭시워치4 40mm 실버 블루투스',
 '워치4 미개봉 갤럭시워치4 40mm 실버 블루투스',
 '40mm 미개봉 갤럭시워치4 40mm 실버 블루투스',
 'watch 미개봉 갤럭시워치4 40mm 실버 블루투스',
 '105 갤럭시   구스 105',
 '갤럭시 갤럭시 s20 플러스 쥐색 회색',
 's20 갤럭시 s20 플러스 쥐색 회색',
 '플러스 갤럭시 s20 플러스 쥐색 회색',
 '미밴드  미개봉 갤럭시워치4 그린 44mm',
 '애플워치  미개봉 갤럭시워치4 그린 44mm',
 '갤럭시워치  미개봉 갤럭시워치4 그린 44mm',
 '순토  미개봉 갤럭시워치4 그린 44mm',
 '버즈 갤럭시 버즈 프로 미개봉 새제품 판매합니다',
 '버즈프로 갤럭시 버즈 프로 미개봉 새제품 판매합니다',
 '버즈2 갤럭시 버즈 프로 미개봉 새제품 판매합니다',
 '버즈라이브 갤럭시 버즈 프로 미개봉 새제품 판매합니다',
 '에어팟 갤럭시 버즈 프로 미개봉 새제품 판매합니다',
 '폴더2 삼성 갤럭시 폴더2 G160 새제품 판매합니다',
 'G160 삼성 갤럭시 폴더2 G160 새제품 판매합니다',
 '폴더 삼성 갤럭시 폴더2 G160 새제품 판매합니다',
 '폴더폰 삼성 갤럭시 폴더2 G160 새제품 판매합니다',
 '효도폰 삼성 갤럭시 폴더2 G160 새제품 판매합니다',
 '갤럭시A40 갤럭시A905G A908 LG 화이트 128GB 17만 47695',
 '갤럭시A50 갤럭시A905G A908 LG 화이트 128GB 17만 47695',
 '갤럭시J4플러스 갤럭시A905G A908 LG 화이트 128GB 17만 47695',
 '갤럭시J6 갤럭시A905G A908 LG 화이트 128GB 17만 47695',
 '갤럭시A90 갤럭시A905G A908 LG 화이트 128GB 17만 47695',
 '워치 겔럭시기어 워치4 클래식 46mm 팔아요  ',
 '겔럭시기어 겔럭시기어 워치4 클래

## 띄어쓰기를 기준으로 문장 분리


In [ ]:
train_sentences[1]

'워치 미개봉 갤럭시워치4 40mm 실버 블루투스'

In [ ]:
len(train_sentences[1].split())

6

In [ ]:
key_list = []
for i in range(len(train_sentences)):
    l = []
    for j in range(len(train_sentences[i].split())):
        l.append(train_sentences[i].split()[j])
    key_list.append(l)

key_list

[['갤럭시', '미개봉', '갤럭시워치4', '40mm', '실버', '블루투스'],
 ['워치', '미개봉', '갤럭시워치4', '40mm', '실버', '블루투스'],
 ['워치4', '미개봉', '갤럭시워치4', '40mm', '실버', '블루투스'],
 ['40mm', '미개봉', '갤럭시워치4', '40mm', '실버', '블루투스'],
 ['watch', '미개봉', '갤럭시워치4', '40mm', '실버', '블루투스'],
 ['105', '갤럭시', '구스', '105'],
 ['갤럭시', '갤럭시', 's20', '플러스', '쥐색', '회색'],
 ['s20', '갤럭시', 's20', '플러스', '쥐색', '회색'],
 ['플러스', '갤럭시', 's20', '플러스', '쥐색', '회색'],
 ['미밴드', '미개봉', '갤럭시워치4', '그린', '44mm'],
 ['애플워치', '미개봉', '갤럭시워치4', '그린', '44mm'],
 ['갤럭시워치', '미개봉', '갤럭시워치4', '그린', '44mm'],
 ['순토', '미개봉', '갤럭시워치4', '그린', '44mm'],
 ['버즈', '갤럭시', '버즈', '프로', '미개봉', '새제품', '판매합니다'],
 ['버즈프로', '갤럭시', '버즈', '프로', '미개봉', '새제품', '판매합니다'],
 ['버즈2', '갤럭시', '버즈', '프로', '미개봉', '새제품', '판매합니다'],
 ['버즈라이브', '갤럭시', '버즈', '프로', '미개봉', '새제품', '판매합니다'],
 ['에어팟', '갤럭시', '버즈', '프로', '미개봉', '새제품', '판매합니다'],
 ['폴더2', '삼성', '갤럭시', '폴더2', 'G160', '새제품', '판매합니다'],
 ['G160', '삼성', '갤럭시', '폴더2', 'G160', '새제품', '판매합니다'],
 ['폴더', '삼성', '갤럭시', '폴더2', 'G160', '새제품', '판매합니다'],
 ['폴

In [ ]:
# 첫 문장 확인
key_list[0]

In [ ]:
### word2vec 학습에 맞는 형태로 데이터 저장

from typing import List
from tqdm import tqdm

def write_to_txt(sentences,
                 filename):
    """
    전처리 완료된 데이터를 txt파일로 저장하는 함수
    :param sentences: 전처리 완료된 문장 리스트
    :param filename: 저장 파일 경로
    """
    with open(filename, 'w', encoding='utf-8') as f:
        for sent in tqdm(sentences):
            for i in sent[:-1]:
                f.write(i + ' ')
            f.write(sent[-1])
            f.write('\n')
    f.close()
    print(f'Data saved at {filename}')

In [ ]:
write_to_txt(key_list, os.path.join(path, 'keyboard_train.txt'))

100%|██████████| 19598/19598 [00:00<00:00, 424472.61it/s]

Data saved at /content/drive/MyDrive/Colab Notebooks/tag/keyboard_train.txt


## word2vec 문장 데이터 불러오기

In [ ]:
class SentenceReader:
    def __init__(self, filepath):
        self.filepath = filepath

    def __iter__(self):
        for line in open(self.filepath):
            yield line.split(' ')

In [ ]:
class SentenceReader:
    def __init__(self, filepath):
        self.filepath = filepath
    def __iter__(self):
        for line in open(self.filepath):
            yield line.replace('\n','').split(' ')

In [ ]:
sentences = SentenceReader(os.path.join(path, 'keyboard_train.txt'))

In [ ]:
print(sentences)

## 많이 등장하는 단어 확인

In [ ]:
from collections import defaultdict

word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

7544

In [ ]:
sorted(word_freq, key=word_freq.get, reverse=True)[:20]

['갤럭시',
 '블랙',
 '판매합니다',
 '삼성',
 '중고폰',
 '팝니다',
 '5G',
 '256GB',
 'A급',
 '256기가',
 '공기계',
 '화이트',
 '케이스',
 '무잔상',
 '판매',
 '갤럭시노트10',
 '128GB',
 '미개봉',
 '64GB',
 '갤럭시S10']

## word2vec 모델 생성 및 확습

In [ ]:
import multiprocessing
from gensim.models import Word2Vec

In [ ]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

In [ ]:
cores

4

In [ ]:
w2v_model = Word2Vec(min_count=40,
                     window=5,
                     size=300,
                     sample=6e-5, 
                     alpha=0.01, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

### 참고
### parameter

* `min_count` <font color='purple'>=</font> <font color='green'>int</font> - Ignores all words with total absolute frequency lower than this - (2, 100)


* `window` <font color='purple'>=</font> <font color='green'>int</font> - The maximum distance between the current and predicted word within a sentence. E.g. `window` words on the left and `window` words on the left of our target - (2, 10)


* `vector_size` <font color='purple'>=</font> <font color='green'>int</font> - Dimensionality of the feature vectors. - (50, 300)


* `sample` <font color='purple'>=</font> <font color='green'>float</font> - The threshold for configuring which higher-frequency words are randomly downsampled. Highly influencial.  - (0, 1e-5)


* `alpha` <font color='purple'>=</font> <font color='green'>float</font> - The initial learning rate - (0.01, 0.05)


* `min_alpha` <font color='purple'>=</font> <font color='green'>float</font> - Learning rate will linearly drop to `min_alpha` as training progresses. To set it: alpha - (min_alpha * epochs) ~ 0.00


* `negative` <font color='purple'>=</font> <font color='green'>int</font> - If > 0, negative sampling will be used, the int for negative specifies how many "noise words" should be drown. If set to 0, no negative sampling is used. - (5, 20)


* `workers` <font color='purple'>=</font> <font color='green'>int</font> - Use these many worker threads to train the model (=faster training with multicore machines)

### Build Vocab

In [ ]:
from time import time
t = time()
w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

2022-01-11 04:20:30,118 : INFO : collecting all words and their counts
2022-01-11 04:20:30,121 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-01-11 04:20:30,148 : INFO : PROGRESS: at sentence #10000, processed 63402 words, keeping 4371 word types
2022-01-11 04:20:30,173 : INFO : collected 7544 word types from a corpus of 119847 raw words and 19598 sentences
2022-01-11 04:20:30,174 : INFO : Loading a fresh vocabulary
2022-01-11 04:20:30,178 : INFO : effective_min_count=40 retains 438 unique words (5% of original 7544, drops 7106)
2022-01-11 04:20:30,179 : INFO : effective_min_count=40 leaves 83394 word corpus (69% of original 119847, drops 36453)
2022-01-11 04:20:30,182 : INFO : deleting the raw counts dictionary of 7544 items
2022-01-11 04:20:30,183 : INFO : sample=6e-05 downsamples 438 most-common words
2022-01-11 04:20:30,184 : INFO : downsampling leaves estimated 13551 word corpus (16.2% of prior 83394)
2022-01-11 04:20:30,186 : INFO : estimated requ

Time to build vocab: 0.0 mins


### 모델학습


In [ ]:
t = time()
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=10, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

2022-01-11 04:20:32,924 : INFO : training model with 3 workers on 438 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=5
2022-01-11 04:20:33,018 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-01-11 04:20:33,022 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-01-11 04:20:33,023 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-01-11 04:20:33,024 : INFO : EPOCH - 1 : training on 119847 raw words (13640 effective words) took 0.1s, 143161 effective words/s
2022-01-11 04:20:33,110 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-01-11 04:20:33,116 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-01-11 04:20:33,118 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-01-11 04:20:33,121 : INFO : EPOCH - 2 : training on 119847 raw words (13680 effective words) took 0.1s, 148023 effective words/s
2022-01-11 04:20:33,212 : INFO : work

Time to train the model: 0.02 mins


In [ ]:
# 모델을 더 이상 학습시키지 않는다면, init_sims로 모델을 memory-efficient하게 만들 수 있다
w2v_model.init_sims(replace=True)

2022-01-11 04:20:34,199 : INFO : precomputing L2-norms of word weight vectors


In [ ]:
w2v_model.save(os.path.join(path, 'word2vec_model'))

2022-01-11 04:20:38,229 : INFO : saving Word2Vec object under /content/drive/MyDrive/Colab Notebooks/tag/word2vec_model, separately None
2022-01-11 04:20:38,231 : INFO : not storing attribute vectors_norm
2022-01-11 04:20:38,233 : INFO : not storing attribute cum_table
2022-01-11 04:20:38,766 : INFO : saved /content/drive/MyDrive/Colab Notebooks/tag/word2vec_model


## 생성된 모델에 단어 입력

In [ ]:
from gensim.models import KeyedVectors
w2v_model = KeyedVectors.load(os.path.join(path, 'word2vec_model'), mmap='r')

2022-01-11 04:20:40,709 : INFO : loading Word2VecKeyedVectors object from /content/drive/MyDrive/Colab Notebooks/tag/word2vec_model
2022-01-11 04:20:40,723 : INFO : loading wv recursively from /content/drive/MyDrive/Colab Notebooks/tag/word2vec_model.wv.* with mmap=r
2022-01-11 04:20:40,724 : INFO : setting ignored attribute vectors_norm to None
2022-01-11 04:20:40,725 : INFO : loading vocabulary recursively from /content/drive/MyDrive/Colab Notebooks/tag/word2vec_model.vocabulary.* with mmap=r
2022-01-11 04:20:40,728 : INFO : loading trainables recursively from /content/drive/MyDrive/Colab Notebooks/tag/word2vec_model.trainables.* with mmap=r
2022-01-11 04:20:40,730 : INFO : setting ignored attribute cum_table to None
2022-01-11 04:20:40,732 : INFO : loaded /content/drive/MyDrive/Colab Notebooks/tag/word2vec_model


In [ ]:
w2v_model.wv.most_similar(positive=["갤럭시"])

2022-01-11 04:20:42,259 : INFO : precomputing L2-norms of word weight vectors


[('S급', 0.9999469518661499),
 ('미개봉', 0.9999412298202515),
 ('갤럭시노트10', 0.9999412298202515),
 ('블랙', 0.9999405145645142),
 ('할인', 0.9999403953552246),
 ('상품', 0.9999403953552246),
 ('참조', 0.999940037727356),
 ('케이스', 0.9999399185180664),
 ('방탄소년단', 0.9999399185180664),
 ('SK', 0.9999396800994873)]

## Word2Vec으로 태그 추출

In [ ]:
test_sentences = input() 

In [ ]:
t = []
for word in api.analyze(test_sentences):
            for morph in word.morphs:
                if morph.tag in ['NNG', 'NNP']:
                    t.append(morph.lex)

t

In [ ]:
# 생성된 word2vec 모델로 입력된 문장과 유사한 단어 top5
w2v_model.wv.most_similar(positive=['갤럭시', '노트20', '중고', '팝니다'], topn=5)